In [122]:
#Packages needed
using Optim
using Plots
using StatsBase
using Distributions
using Statistics
using Random
using JuMP, Gurobi, GLPK
using TickTock
using Distributed
using CSV
using DataFrames
using JLD2
using Dates

####### This function just checks the OS and helps automate things instead of needed to copy and paste paths
function checkOS()
    path = pwd()
    if path[1] == '/'
        OS = "Mac"
    else
        OS = "Windows"
    end
    return OS
end
OS = checkOS()
file_path = pwd();

In [123]:
#####Load correct .jl files for functions (accommodates for Mac or Windows)
if OS == "Mac"
    inipath = file_path[1:findlast("Julia_Code",file_path)[end]]*"/"
    include(inipath * "1. Tradeoff Curvers and Value of AM Addition/2. Scripts/Generate_Forecast.jl")
    include(inipath * "3. Functions/MILPSetups.jl")
    include(inipath * "3. Functions/Functions.jl")
else 
    inipath = file_path[1:findlast("Julia_Code",file_path)[end]] * "\\"
    include(inipath * "1. Tradeoff Curvers and Value of AM Addition\\2. Scripts\\Generate_Forecast.jl")
    include(inipath * "3. Functions\\MILPSetups.jl")
    include(inipath * "3. Functions\\Functions.jl")
end

save_data

In [141]:
###########Test Conditions
avg_dem = 3
adi = 4.5
cv2 = 4.5
lev = avg_dem * adi
T = 60      #Time Horizon
G = 1 #Number of products
obs = 180
nreal = 100 # number of realizations to generate
LT = [7] #based on LT of resupply from TM to supplier
N=1000 # Number of replications to generate LTD based on
AM_capacity = 4

4

In [142]:
############Supply Chain Design
"""
#Kyle's paper matrix 1-3-2

trans_matrix = [0 25 25 25 0  0;
                0 0  0  0  25 0;
                0 0  0  0  50 50;
                0 0  0  0  0  25;
                0 0  0  0  0  0;
                0 0  0  0  0  0]
TM = [1]
DC = [2 3 4]
SL = [5 6]
"""
#Validation SC 1-1-4
trans_matrix = [0 25 0  0  0  0;
                0 0  25 25 25 25;
                0 0  0  0  0  0;
                0 0  0  0  0  0;
                0 0  0  0  0  0;
                0 0  0  0  0  0]

TM = [1]
DC = [2]
SL = [3 4 5 6]
"""
#Network - 1-3-6 Iso
trans_matrix = [0 25 25 25  0  0  0  0  0  0; #will vary the last 25 in this row
                0 0  0  0  25 25 25  0  0  0;
                0 0  0  0  0  25 25 25  0  0;
                0 0  0  0  0  0   0  0 25 25;
                0 0  0  0  0  0   0  0  0  0;
                0 0  0  0  0  0   0  0  0  0;
                0 0  0  0  0  0   0  0  0  0;
                0 0  0  0  0  0   0  0  0  0;
                0 0  0  0  0  0   0  0  0  0;
                0 0  0  0  0  0   0  0  0  0]
TM = [1]
DC = [2 3 4]
SL = [5 6 7 8 9 10]
"""

nsup, ncust, nTM, valid_TMrte, Suprte, CtTM, CtS = trans_matrix_math(trans_matrix,TM,DC,SL)

(1, 4, 1, [1.0], [1.0 1.0 1.0 1.0], [25.0], [25.0 25.0 25.0 25.0])

In [143]:
######### All generate forecast code
hist_dem = gen_historical(adi, cv2, lev, G, nsup)
DC_dem, baseline_dem = gen_dem_realization(nreal, adi, cv2, lev, T, G, nsup);
dem_real = split_demand(nreal,T,ncust,nsup,G,Suprte,DC_dem)
forecast = forecast_dem(hist_dem,G,T,ncust,Suprte)
(LTD, Q, pAll) = little_s_inventory_DC(nsup, ncust, LT, N, hist_dem,true)
NetDesign = AM_design(nsup,ncust)
ndesign = size(NetDesign)[1]

"""
##### Validation code, done to bring MATLAB data over
using DataFrames; forecast = CSV.read("C:\\Users\\lmdoudni\\Documents\\forecast.csv", DataFrame)
using DataFrames; hist_dem = CSV.read("C:\\Users\\lmdoudni\\Documents\\hist_dem.csv", DataFrame)
using DataFrames; demand = CSV.read("C:\\Users\\lmdoudni\\Documents\\demand.csv", DataFrame)
forecast = Matrix(forecast)
forecast = reshape(forecast,(2,60,3))
hist_dem = Matrix(hist_dem)
hist_dem = reshape(hist_dem,(2,180,3))
demand = Matrix(demand)
hold = zeros(50,60,2,3)
for g=1:G
    for t=1:T
        hold[:,t,1:2,g] = demand[:,((2*t*g)-1):(2*t*g)]
    end
end
dem_real = floor.(Int64,hold)

#Hard Coded Min Policy
pAll = [18 19 19; 18 19 19; 18 19 19]
"""

idx_Croston = Int64[]
idx_SBA = [1]


"##### Validation code, done to bring MATLAB data over\nusing DataFrames; forecast = CSV.read(\"C:\\Users\\lmdoudni\\Documents\\forecast.csv\", DataFrame)\nusing DataFrames; hist_dem = CSV.read(\"C:\\Users\\lmdoudni\\Documents\\hist_dem.csv\", DataFrame)\nusing DataFrames; demand = CSV.read(\"C:\\Users\\lmdoudni\\Documents\\demand.csv\", DataFrame)\nforecast = Matrix(forecast)\nforecast = reshape(forecast,(2,60,3))\nhist_dem = Matrix(hist_dem)\nhist_dem = reshape(hist_dem,(2,180,3))\ndemand = Matrix(demand)\nhold = zeros(50,60,2,3)\nfor g=1:G\n    for t=1:T\n        hold[:,t,1:2,g] = demand[:,((2*t*g)-1):(2*t*g)]\n    end\nend\ndem_real = floor.(Int64,hold)\n\n#Hard Coded Min Policy\npAll = [18 19 19; 18 19 19; 18 19 19]\n"

In [149]:
#Cost and Lag Inputs (Edit if you really want to)
##### NOTE: you can input the exact dimensions and change cost as you please or you can have just a single number there and that will address all the variation in check dims command
LagTM = LT # Rows = TM Columns = Sup 

LagS = [2] # Rows = Sup  Columns = Cust 


CpTM = [1] #rows = num products

#********CpAM incorporates both supplier and customers becasue AM is located at both locs
CpAM = [1] #Rows = Sup and Cust, Sups first, Cols = Time, 3D=Prod  

##### This might be needed later
CtSexpanded = repeat(CtS,1,T,G)

#All needs to be a function of value of the product. Wont be the same for all products usually
Ci = [3] #Rows = Sup, Cols = Time, 3D = prod

CiRetailer = [5] #rows = Retailer, Cols = Time, 3D = prod
CBO = [378] #Rows = Retailer, Cols = Time, 3D = prod


##### Setting up expedited infrastructure
CpExp = ones(1,T,G) #covers all expedited prod...... Rows = nTM, Cols = Time, 3D = prod

### cost of expedited teransportation. TM direct to SL
CtExp = [100] #Transport from RETAILER DIRECT location. Rows = RETAILER Locs, Cols =Time (in the end), 3D Prod (new TM)

LagExp = [3] #Rows = TM, Cols = Retailers


###### Transhipment  ********* NEEDS WORK ON LATER
#Expedited transport cost from Supplier to Supplier. ONLY USED IN EVAL MILP
CtransferExp = [0 25 25; 25 0 25; 25 25 0] #square matrix of ALL suppliers. cost of lateral transfer between suppliers

#valid_transferExprte = [0 1 1; 1 0 1; 1 1 0]
#valid_transferExprte = [0 0 0; 0 0 0; 0 0 0]

LagTransferExp = 1 #time lag to transfer between suppliers (ASSUME ALL LAGS ARE 1)

#The transferred prodcut still flows to the customer in the same manner as normal product. Break it out in separate vars for visualization
CtransferExpFLOW = CtSexpanded

#AM stuff must incorporate both suppliers and customers because both locs have AM, 
AM_fixed_cost = [121] #suppliers must come first #this is per day

#Suppliers and cust. Suppliers must come first
K_stable = [1] #AM indicator. Rows = Sup and Cust. Cols = Time. 3D=Products  ####used to be capacity but i made that a scalar above

#Expedited Capacity (can change to help prevent eval model from expediting everything)
CtExpCapacity = zeros(size(CtExp))
CtExpCapacity .= 0 #change as needed

yinit = pAll .+ 10 #Start with more products?
#LagTM,LagS,CpTM,valid_TMrte,valid_Suprte,Ci,CiRetailer,CBO,CpAM,AM_fixed_cost,K_stable = check_dims(nsup,ncust,nTM,T,G,LagTM,LagS,CpTM,valid_TMrte,Suprte,Ci,CiRetailer,CBO,CpAM,AM_fixed_cost,K_stable)

1×1 Matrix{Float64}:
 61.0

In [150]:
tick()
Big_S,TC_stored = big_S_MILP(T,G,nsup,ncust,CpTM,CpAM,CtTM,CtS,Ci,CiRetailer,CBO,valid_TMrte,Suprte,CtExp,CtransferExp,K_stable,LagS,
    LagTM,ndesign,AM_fixed_cost,LagTransferExp,CtransferExpFLOW, yinit, pAll, forecast, NetDesign, AM_capacity)
tock()

Design 1

┌ Info:  started timer at: 2021-11-11T18:06:10.869
└ @ TickTock C:\Users\lmdoudni\.julia\packages\TickTock\RsTHR\src\TickTock.jl:32



[36.0 0.0 36.0 0.0 0.0 36.0 0.0 0.0 0.0 0.0 0.0 37.99999999999994 0.0 0.0 0.0 0.0 0.0 36.0 0.0 0.0 0.0 0.0 0.0 36.0 0.0 0.0 0.0 0.0 0.0 36.0 0.0 0.0 0.0 0.0 0.0 36.0 0.0 0.0 0.0 0.0 0.0 36.00000000000182 0.0 0.0 0.0 0.0 0.0 36.0 0.0 0.0 0.0 0.0 0.0 35.99999999999818 0.0 0.0 0.0 0.0 0.0 0.0]
Design 2
[35.0 0.0 35.0 0.0 0.0 0.0 0.0 0.0 37.0 0.0 0.0 0.0 0.0 0.0 36.0 0.0 0.0 0.0 0.0 0.0 35.0 0.0 0.0 0.0 0.0 0.0 35.0 0.0 0.0 0.0 0.0 0.0 35.0 0.0 0.0 0.0 0.0 0.0 36.0 0.0 0.0 0.0 0.0 0.0 35.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
Design 3
[29.0 0.0 29.0 0.0 0.0 0.0 41.0 0.0 0.0 0.0 0.0 0.0 35.0 0.0 0.0 0.0 0.0 0.0 29.0 0.0 0.0 0.0 0.0 0.0 29.0 0.0 0.0 0.0 0.0 0.0 30.0 0.0 0.0 0.0 0.0 0.0 29.0 0.0 0.0 0.0 0.0 0.0 29.0 0.0 0.0 0.0 0.0 0.0 29.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
Design 4
[22.0 0.0 22.0 0.0 0.0 0.0 33.0 0.0 0.0 0.0 0.0 0.0 27.00000000000044 0.0 0.0 0.0 0.0 0.0 22.0 0.0 0.0 0.0 0.0 0.0 22.0 0.0 0.0 0.0 0.0 22.0 0.0 0.0 0.0 0.0 0.0 22.0 0.0 0.0 0.0

┌ Info:           84.3219939s: 1 minute, 24 seconds, 321 milliseconds
└ @ TickTock C:\Users\lmdoudni\.julia\packages\TickTock\RsTHR\src\TickTock.jl:39


In [151]:
pAll

1×1 Matrix{Float64}:
 51.0

In [152]:
permutedims(Big_S,[3,1,2]) #pAll

32×1×1 Array{Float64, 3}:
[:, :, 1] =
 87.0
 86.0
 80.0
 73.0
 83.99999999999999
 72.0
 72.0
 73.0
 85.0
 72.0
 72.0
 73.0
 71.0
  ⋮
 69.0
 72.0
 71.99999999999997
 66.0
 70.0
 72.0
 73.0
 64.0
 71.0
 68.00000000000001
 65.0
 87.0

In [140]:
tick()
stored_TC, stored_delivered, stored_BOtotal, stored_BOtrue, stored_BOlag, stored_AMprod, stored_transDC, stored_TMprodCost, 
    stored_TMtransCost, stored_SupTransCost, stored_InvCarCost, stored_AMprodcost, stored_reorder_qty, stored_transDC, stored_supinv, 
    stored_invDC, stored_reorder_delivered  = MILP2(T,G,nsup, ncust,CpTM,CpAM,CtTM,CtS,Ci,CiRetailer,CBO,valid_TMrte,Suprte,CtExp,
    CtransferExp,K_stable,LagS,LagTM,ndesign, AM_fixed_cost,LagTransferExp,CtransferExpFLOW, yinit, pAll, dem_real, NetDesign, 
    AM_capacity, Big_S) 
tock()

Design 1

┌ Info:  started timer at: 2021-11-11T16:42:00.682
└ @ TickTock C:\Users\lmdoudni\.julia\packages\TickTock\RsTHR\src\TickTock.jl:32



Design 2
Design 3
Design 4
Design 5
Design 6
Design 7
Design 8
Design 9
Design 10
Design 11
Design 12
Design 13
Design 14
Design 15
Design 16
Design 17
Design 18
Design 19
Design 20
Design 21
Design 22
Design 23
Design 24
Design 25
Design 26
Design 27
Design 28
Design 29
Design 30
Design 31
Design 32


┌ Info:          141.7237174s: 2 minutes, 21 seconds, 723 milliseconds
└ @ TickTock C:\Users\lmdoudni\.julia\packages\TickTock\RsTHR\src\TickTock.jl:39


In [51]:
TC,BOtot, AMproduced, TMprodCost, TMtransCost, SupTransCost, InvCarCost, AMprodCost, percent_byAM, TCstdev, BOstdev, BO_by_SL = analyze_outputs(stored_TC,stored_delivered,
    stored_BOtrue,stored_AMprod,stored_transDC, stored_TMprodCost, stored_TMtransCost, stored_SupTransCost, stored_InvCarCost, stored_AMprodcost,dem_real)

([95864.1; 80697.7; … ; 107483.4; 107412.0], [7.5; 0.0; … ; 0.0; 0.0], [0.0]

[460.5]

[451.3]

...

[347.7]

[341.7]

[485.0], [1698.1; 1207.9; … ; 1324.6; 1185.5], [37452.5; 26762.5; … ; 29162.5; 25915.0], [44550.0; 31252.5; … ; 34507.5; 29482.5], [12163.5; 13754.3; … ; 13107.1; 14044.0], [0.0; 460.5; … ; 341.7; 485.0], [0.0; 0.28262831833782986; … ; 0.20971560831864763; 0.2976881358428374], [1444.8669642719512; 1730.561825920511; … ; 1457.6732753871074; 1292.192538113247], [15.953056133543816; 0.0; … ; 0.0; 0.0], [5.4 2.1; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0])

In [52]:
BO_by_SL

32×2×1×1×1 Array{Float64, 5}:
[:, :, 1, 1, 1] =
 5.4  2.1
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.9  0.0
 1.4  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.5  0.0
 ⋮    
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

In [55]:
description = "134Net_Iso_Initial_Tests_AMCap_4_Erratic"
save_data(description,adi,cv2,avg_dem,TC, BOtot, AMproduced, TMprodCost, TMtransCost, SupTransCost, InvCarCost, AMprodCost, Big_S, pAll, percent_byAM, TCstdev, BOstdev, BO_by_SL)

C:\Users\lmdoudni\Documents\Julia_Code\9. Raw Output Data\2021-09-10_134Net_Iso_Initial_Tests_AMCap_4_Erratic_1_2ADI_0_75CV2_3AvgDem.jld2


"Data saved here: C:\\Users\\lmdoudni\\Documents\\Julia_Code\\9. Raw Output Data\\2021-09-10_134Net_Iso_Initial_Tests_AMCap_4_Erratic_1_2ADI_0_75CV2_3AvgDem.jld2"

In [56]:
stored_invDC[1,:,1,1,32]

67-element Vector{Float64}:
  0.0
 38.0
 38.0
 35.0
 32.0
 32.0
 32.0
 32.0
 31.0
 29.0
 28.0
 36.0
 34.0
  ⋮
 15.0
 15.0
 15.0
 12.0
 38.0
 38.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [67]:
display("text/plain", sum(stored_invDC[:,:,1,1,32],dims=1)')

67×1 adjoint(::Matrix{Float64}) with eltype Float64:
   0.0
 109.0
 105.0
  99.0
  91.0
  91.0
  89.0
  74.0
  72.0
  77.0
  63.0
  85.0
  81.0
   ⋮
  64.0
  61.0
  57.0
  61.0
  87.0
 104.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0

In [66]:
sum(sum(stored_transDC[:,:,1,:,1,32],dims=1),dims=3)

1×60×1 Array{Float64, 3}:
[:, :, 1] =
 3.0  4.0  6.0  8.0  0.0  2.0  15.0  …  14.0  11.0  3.0  4.0  6.0  5.0  3.0

In [78]:
std(sum(sum(sum(stored_BOtrue,dims=1),dims=2),dims=3),dims=4)

1×1×1×1×32 Array{Float64, 5}:
[:, :, 1, 1, 1] =
 2.6029810226126564

[:, :, 1, 1, 2] =
 0.0

[:, :, 1, 1, 3] =
 0.0

...

[:, :, 1, 1, 30] =
 0.0

[:, :, 1, 1, 31] =
 0.0

[:, :, 1, 1, 32] =
 0.0

In [81]:
permutedims(std(sum(stored_TC,dims=1),dims=2),[3,1,2])

32×1×1 Array{Float64, 3}:
[:, :, 1] =
 1401.309766917059
 1413.430029743104
 1193.6711604054417
 1048.3569660012115
 1534.3043739221805
 1188.5880989558964
 1304.9211578341678
 1251.5595282099293
 1495.9418002702907
 1168.1485404917737
 1376.66911768654
 1203.5036065672625
 1373.4341408249882
    ⋮
 1374.8937658404516
 1117.3361075087735
 1049.893522297983
 1096.9513739153135
 1343.6083505217719
 1129.4303547020722
 1421.7842914551318
 1028.5999180578285
 1255.6434538774556
 1032.1684740271378
 1289.4562829791032
 1005.8513804692205